# Oregon Old Growth

This notebook records that GDAL commands used to generate the layers used in the oregon.anmiller.com.

## Layers

### Oregon State Boundary

Vector, only simplified

### Private Land in Oregon

Taken from GNN raster, then vectorized.

### Old growth in oregon

Taken from LEMMA, then LSOG classified, then raster compressed.

In [ ]:
"""
Install libraries:

pip install folium
pip install geopandas
pip install rasterio
pip install gdal
"""

# Import necessary libraries

In [ ]:
import folium
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
import numpy as np
import matplotlib.pyplot as plt



In [ ]:

# Define the center of the map
center = [44.0, -120.5]

# Create the map object
m = folium.Map(location=center, zoom_start=7, tiles=None)

# Add a Mapbox satellite layer
mapbox_access_token = 'your_mapbox_access_token_here'
mapbox_tiles = 'https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/{z}/{x}/{y}?access_token=' + mapbox_access_token
folium.TileLayer(
    tiles=mapbox_tiles,
    attr='Mapbox',
    name='Mapbox Satellite',
    overlay=True,
    control=True
).add_to(m)

In [ ]:

# Load the GeoJSON for Oregon's boundaries
oregon_geojson = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [-124.566244, 42.000709],
                [-123.382373, 42.000709],
                [-123.382373, 46.258633],
                [-124.566244, 46.258633],
                [-124.566244, 42.000709]
            ]]
        },
        "properties": {
            "name": "Oregon"
        }
    }]
}

# Convert GeoJSON to GeoDataFrame
oregon_gdf = gpd.GeoDataFrame.from_features(oregon_geojson['features'])

# Path to the raster file
raster_path = 'path_to_your_raster_file.tif'

# Open the raster file
with rasterio.open(raster_path) as src:
    # Clip the raster with the GeoDataFrame boundary
    out_image, out_transform = mask(src, [mapping(oregon_gdf.geometry[0])], crop=True)
    out_meta = src.meta

# Update the metadata with new dimensions, transform, and CRS
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# Save the clipped raster to a new file
clipped_raster_path = 'clipped_raster.tif'
with rasterio.open(clipped_raster_path, 'w', **out_meta) as dest:
    dest.write(out_image)

# Add the GeoJSON layer to the map
folium.GeoJson(
    oregon_geojson,
    name='Oregon Boundary'
).add_to(m)

# Add layer control to switch between layers
folium.LayerControl().add_to(m)

# Display the map
m

gdal:rasterize --distance_units=meters --area_units=m2 --ellipsoid=EPSG:7030 --INPUT=/Users/am/Desktop/oregon/final/simplified-private-land.geojson --FIELD= --BURN=1 --USE_Z=false --UNITS=0 --WIDTH=2000 --HEIGHT=2000 --EXTENT='-124.703726901,-116.463106651,41.992081039,46.292043341 [EPSG:4326]' --NODATA=0 --OPTIONS= --DATA_TYPE=5 --INVERT=false --EXTRA= --OUTPUT=TEMPORARY_OUTPUT

In [ ]:
gdal:rasterize --distance_units=meters --area_units=m2 --ellipsoid=EPSG:7030 --INPUT=/Users/am/Desktop/oregon/final/simplified-private-land.geojson --FIELD= --BURN=1 --USE_Z=false --UNITS=0 --WIDTH=2000 --HEIGHT=2000 --EXTENT='-124.703726901,-116.463106651,41.992081039,46.292043341 [EPSG:4326]' --NODATA=0 --OPTIONS= --DATA_TYPE=5 --INVERT=false --EXTRA= --OUTPUT=TEMPORARY_OUTPUT

processing.run("gdal:rasterize", {'INPUT':'/Users/am/Desktop/oregon/final/simplified-private-land.geojson','FIELD':'','BURN':1,'USE_Z':False,'UNITS':0,'WIDTH':2000,'HEIGHT':2000,'EXTENT':'-124.703726901,-116.463106651,41.992081039,46.292043341 [EPSG:4326]','NODATA':0,'OPTIONS':'','DATA_TYPE':5,'INIT':None,'INVERT':False,'EXTRA':'','OUTPUT':'TEMPORARY_OUTPUT'})